In [1]:
import pandas as pd, json as json

In [2]:
ds = pd.read_csv('../A8/cp255A8.csv')

In [3]:
ds.head()

,Unnamed: 0,UID,IsBridge,SURFTHK,ULT,MinT,MaxT,prep,AADT,Y_ESAL,Longitude,Latitude
0,712,1,0,0.25,1.00,-0.69,24.900000,841.650024,6415,30058,-122.169366,47.978215
1,635,24,1,0.00,0.15,-0.92,24.650000,906.809998,45712,174012,-122.133338,47.978414
2,671,33,0,0.12,1.54,-0.92,24.650000,906.809998,31066,118246,-122.120592,47.970205
3,27,66,1,0.00,0.15,-1.14,24.420000,1024.199951,20944,79442,-122.070079,47.947539
4,694,81,0,0.25,1.15,-1.31,24.799999,1088.739990,20995,79671,-122.062374,47.922928


In [4]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
ds['Latitude'] = ds['Latitude'].astype(float)
ds['Longitude'] = ds['Longitude'].astype(float)
ds['prep'] = ds['prep'].astype(float)

In [6]:
# removing some columns
cols = ['IsBridge', 'ULT', 'Latitude', 'Longitude', 'MinT', 'MaxT', 'prep', 'Y_ESAL']
df = ds[cols]

In [8]:
print len(df[df.Longitude.isnull() == 1])

0


In [23]:
# create a new python dict to contain our geojson data, using geojson format
geojson = {'type':'FeatureCollection', 'features':[]}

In [24]:
# loop through each row in the dataframe and convert each row to geojson format
for _, row in df.iterrows():
    feature = {'type':'Feature',
               'geometry':{
                   'type':'Point',
                   'coordinates':[row['Longitude'],row['Latitude']]},
               'properties': {
                   'layer_thickness':'{} ft thick'.format(round(row['ULT'],2)),
                   'Annual_loading':'{} ESALs'.format(int(row['Y_ESAL'])), 
                   'IsBridge':row['IsBridge'],
                   'Annual_precipitation':'Precipitation: {}'.format(int(row['prep'])),
                    'min_temperature':'Min Temp: {}°C'.format(round(row['MinT'], 2)),
                    'max_temperature':'Max Temp: {}°C'.format(round(row['MaxT'], 2))}}
    
    # add this feature (aka, converted dataframe row) to the list of features inside our dict
    geojson['features'].append(feature)

In [25]:
# save the geojson result to a file
output_filename = '../A8/dataset.js'
with open(output_filename, 'wb') as output_file:
    output_file.write('var dataset = ')
    json.dump(geojson, output_file, indent=2)  
    
# how many features did we save to the geojson file?
print '{} geotagged features saved to file'.format(len(geojson['features']))

1761 geotagged features saved to file
